### sql문 안에서 사용할 수 있는 함수를 만들 수 있는 장치 
count, split 등등 사용할 수 있었는데 
특수한 경우 사용자가 커스텀 하거나 새롭게 만들어 사용할 수 있음 

In [1]:
from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("udf").getOrCreate()

23/01/20 12:54:36 WARN Utils: Your hostname, imhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.18 instead (on interface en0)
23/01/20 12:54:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/20 12:54:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
transactions = [
    ('찹쌀탕수육+짜장2', '2021-11-07 13:20:00', 22000, 'KRW'),
    ('등심탕수육+크립새우+짜장면', '2021-10-24 11:19:00', 21500, 'KRW'), 
    ('월남 쌈 2인 세트', '2021-07-25 11:12:40', 42000, 'KRW'), 
    ('콩국수+열무비빔국수', '2021-07-10 08:20:00', 21250, 'KRW'), 
    ('장어소금+고추장구이', '2021-07-01 05:36:00', 68700, 'KRW'), 
    ('족발', '2020-08-19 19:04:00', 32000, 'KRW'),  
]

schema = ["name", "datetime", "price", "currency"]

In [3]:
df = spark.createDataFrame(data=transactions, schema=schema)

In [4]:
df.createOrReplaceTempView("transections")

In [5]:
spark.sql("select * from transections").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [14]:
from pyspark.sql.functions import udf



# udf
from pyspark.sql.types import LongType

@udf("long")
def squared(n):
    return n * n

# register 타입을 지정하지 않으면 default string 
# spark.udf.register("squared", squared, LongType())

In [15]:
spark.sql("select price, squared(price) from transections").show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|22000|     484000000|
|21500|     462250000|
|42000|    1764000000|
|21250|     451562500|
|68700|    4719690000|
|32000|    1024000000|
+-----+--------------+



In [16]:
spark.sql("select price, squared(price) from transections").printSchema()

root
 |-- price: long (nullable = true)
 |-- squared(price): long (nullable = true)



In [20]:
def read_number(n):
    units = ["", "십", "백", "천", "만"]
    nums = '일이삼사오육칠팔구'
    result = []
    
    i = 0
    while n > 0:
        n, r = divmod(n, 10)
        if r > 0:
            result.append(nums[r-1]+units[i])
        i += 1
    return "".join(reversed(result))

print(read_number(32000))
print(read_number(68700))

삼만이천
육만팔천칠백


In [21]:
spark.udf.register("read_number", read_number)

<function __main__.read_number(n)>

In [22]:
spark.sql("select price, read_number(price) from transections").show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|22000|          이만이천|
|21500|      이만일천오백|
|42000|          사만이천|
|21250|  이만일천이백오십|
|68700|      육만팔천칠백|
|32000|          삼만이천|
+-----+------------------+



In [23]:
def get_weekday(data):
    import calendar
    return calendar.day_name[data.weekday()]

spark.udf.register("get_weekday", get_weekday)

<function __main__.get_weekday(data)>

In [25]:
qs = """
select
    datetime,
    get_weekday(TO_DATE(datetime)) as day
from
    transections
"""
spark.sql(qs).show()

+-------------------+---------+
|           datetime|      day|
+-------------------+---------+
|2021-11-07 13:20:00|   Sunday|
|2021-10-24 11:19:00|   Sunday|
|2021-07-25 11:12:40|   Sunday|
|2021-07-10 08:20:00| Saturday|
|2021-07-01 05:36:00| Thursday|
|2020-08-19 19:04:00|Wednesday|
+-------------------+---------+



DataPreprocessing 할때 좋음